# Linea de Muerte, Tercera Competencia

### Legal Notice

Alumnos, de la comision del lunes o del jueves

que terminen en el Private Leaderboard por debajo de la ganancia  del submit que genera este muy basico script

tendran severamente comprometidas sus chances de aprobar la asignatura

por favor notar que este script construye todo de cero, y unicamente utiliza conceptos ya que estaban disponibles an la Primera Competencia

### Requerimientos

* Este script necesita apenas una virtual machine con **32 GB** de memoria RAM
* Corre en una hora y cuarenta

## Inicializacion

Establezco Experimento

In [1]:
# limpio la memoria
format(Sys.time(), "%a %b %d %X %Y")
rm(list = ls(all.names = TRUE)) # remove all objects
gc(full = TRUE, verbose= FALSE) # garbage collection

dir.create("~/buckets/b1/exp/lineademuerte/2/", showWarnings = FALSE)
setwd( "~/buckets/b1/exp/lineademuerte/2/" )

[1] "Wed Dec 04 13:32:33 2024"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,655723,35.1,1442488,77.1,1172047,62.6
Vcells,1219423,9.4,8388608,64.0,1924976,14.7


## Creacion clase_ternaria

Creacion de clase_ternaria a partir del dataset crudo

In [2]:
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_03_crudo.csv.gz" )

# calculo el periodo0 consecutivo
setorder( dataset, numero_de_cliente, foto_mes )
dataset[, periodo0 := as.integer(foto_mes/100)*12 +  foto_mes%%100]

# calculo topes
periodo_ultimo <- dataset[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1

# calculo los leads de orden 1 y 2
dataset[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dataset[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dataset[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dataset[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]

dataset[, c("periodo0", "periodo1", "periodo2") := NULL ]

Loading required package: data.table



Verificacion de clase_ternaria

In [3]:
tbl <- dataset[, .N, list(foto_mes, clase_ternaria)]
setorder(tbl, foto_mes, clase_ternaria)
tbl

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,688
201901,BAJA+2,718
201901,CONTINUA,123346
201902,BAJA+1,722
201902,BAJA+2,688
201902,CONTINUA,124389
201903,BAJA+1,689
201903,BAJA+2,760
201903,CONTINUA,124987


## Preprocesamiento

Solamente se hará un Feature Engineering Histórico

Intencionalmente **NO** se hacen
* Data Quality = Catastrophe Analysis   ( variables rotas que están en CERO )
* Data Drifting,  ajuste por Inflacion
* Feature Engineering Manual intra-mes
* Feature Engineering de nuevas variables a partir de hojas de Random Forest

### Feature Engineering Historico

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1    valor actual - lag1
* lag2  lags de orden 2
* delta2  valor actual - lag2

se pasa de  155 campos a 763

(*) Excepto para los campos  [ numero_de_cliente,  foto_mes,  clase_ternaria]

In [4]:
# Feature Engineering Historico
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )


dataset[, 
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

dataset[, 
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags de orden 1
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Veriricacion de los campos recien creados

In [ ]:
ncol(dataset)
colnames(dataset)

In [ ]:
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}
AgregarVariables_IntraMes <- function(dataset) {
  cat( "inicio AgregarVariables_IntraMes()\n")
  gc(verbose= FALSE)
  # INICIO de la seccion donde se deben hacer cambios con variables nuevas

  # el mes 1,2, ..12
  if( atributos_presentes( c("foto_mes") ))
    dataset[, kmes := foto_mes %% 100]

  # creo un ctr_quarter que tenga en cuenta cuando
  # los clientes hace 3 menos meses que estan
  # ya que seria injusto considerar las transacciones medidas en menor tiempo
  if( atributos_presentes( c("ctrx_quarter") ))
    dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[
      cliente_antiguedad == 3,
      ctrx_quarter_normalizado := ctrx_quarter * 1.2
    ]

  # variable extraida de una tesis de maestria de Irlanda
  if( atributos_presentes( c("mpayroll", "cliente_edad") ))
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

  # se crean los nuevos campos para MasterCard  y Visa,
  #  teniendo en cuenta los NA's
  # varias formas de combinar Visa_status y Master_status
  if( atributos_presentes( c("Master_status", "Visa_status") ))
  {
    dataset[, vm_status01 := pmax(Master_status, Visa_status, na.rm = TRUE)]
    dataset[, vm_status02 := Master_status + Visa_status]

    dataset[, vm_status03 := pmax(
      ifelse(is.na(Master_status), 10, Master_status),
      ifelse(is.na(Visa_status), 10, Visa_status)
    )]

    dataset[, vm_status04 := ifelse(is.na(Master_status), 10, Master_status)
      + ifelse(is.na(Visa_status), 10, Visa_status)]

    dataset[, vm_status05 := ifelse(is.na(Master_status), 10, Master_status)
      + 100 * ifelse(is.na(Visa_status), 10, Visa_status)]

    dataset[, vm_status06 := ifelse(is.na(Visa_status),
      ifelse(is.na(Master_status), 10, Master_status),
      Visa_status
    )]

    dataset[, mv_status07 := ifelse(is.na(Master_status),
      ifelse(is.na(Visa_status), 10, Visa_status),
      Master_status
    )]
  }


  # combino MasterCard y Visa
  if( atributos_presentes( c("Master_mfinanciacion_limite", "Visa_mfinanciacion_limite") ))
    dataset[, vm_mfinanciacion_limite := rowSums(cbind(Master_mfinanciacion_limite, Visa_mfinanciacion_limite), na.rm = TRUE)]

  if( atributos_presentes( c("Master_Fvencimiento", "Visa_Fvencimiento") ))
    dataset[, vm_Fvencimiento := pmin(Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE)]

  if( atributos_presentes( c("Master_Finiciomora", "Visa_Finiciomora") ))
    dataset[, vm_Finiciomora := pmin(Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldototal", "Visa_msaldototal") ))
    dataset[, vm_msaldototal := rowSums(cbind(Master_msaldototal, Visa_msaldototal), na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldopesos", "Visa_msaldopesos") ))
    dataset[, vm_msaldopesos := rowSums(cbind(Master_msaldopesos, Visa_msaldopesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldodolares", "Visa_msaldodolares") ))
    dataset[, vm_msaldodolares := rowSums(cbind(Master_msaldodolares, Visa_msaldodolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumospesos", "Visa_mconsumospesos") ))
    dataset[, vm_mconsumospesos := rowSums(cbind(Master_mconsumospesos, Visa_mconsumospesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumosdolares", "Visa_mconsumosdolares") ))
    dataset[, vm_mconsumosdolares := rowSums(cbind(Master_mconsumosdolares, Visa_mconsumosdolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mlimitecompra", "Visa_mlimitecompra") ))
    dataset[, vm_mlimitecompra := rowSums(cbind(Master_mlimitecompra, Visa_mlimitecompra), na.rm = TRUE)]

  if( atributos_presentes( c("Master_madelantopesos", "Visa_madelantopesos") ))
    dataset[, vm_madelantopesos := rowSums(cbind(Master_madelantopesos, Visa_madelantopesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_madelantodolares", "Visa_madelantodolares") ))
    dataset[, vm_madelantodolares := rowSums(cbind(Master_madelantodolares, Visa_madelantodolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_fultimo_cierre", "Visa_fultimo_cierre") ))
    dataset[, vm_fultimo_cierre := pmax(Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagado", "Visa_mpagado") ))
    dataset[, vm_mpagado := rowSums(cbind(Master_mpagado, Visa_mpagado), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagospesos", "Visa_mpagospesos") ))
    dataset[, vm_mpagospesos := rowSums(cbind(Master_mpagospesos, Visa_mpagospesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagosdolares", "Visa_mpagosdolares") ))
    dataset[, vm_mpagosdolares := rowSums(cbind(Master_mpagosdolares, Visa_mpagosdolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_fechaalta", "Visa_fechaalta") ))
    dataset[, vm_fechaalta := pmax(Master_fechaalta, Visa_fechaalta, na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumototal", "Visa_mconsumototal") ))
    dataset[, vm_mconsumototal := rowSums(cbind(Master_mconsumototal, Visa_mconsumototal), na.rm = TRUE)]

  if( atributos_presentes( c("Master_cconsumos", "Visa_cconsumos") ))
    dataset[, vm_cconsumos := rowSums(cbind(Master_cconsumos, Visa_cconsumos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_cadelantosefectivo", "Visa_cadelantosefectivo") ))
    dataset[, vm_cadelantosefectivo := rowSums(cbind(Master_cadelantosefectivo, Visa_cadelantosefectivo), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagominimo", "Visa_mpagominimo") ))
    dataset[, vm_mpagominimo := rowSums(cbind(Master_mpagominimo, Visa_mpagominimo), na.rm = TRUE)]

  # a partir de aqui juego con la suma de Mastercard y Visa
  if( atributos_presentes( c("Master_mlimitecompra", "vm_mlimitecompra") ))
    dataset[, vmr_Master_mlimitecompra := Master_mlimitecompra / vm_mlimitecompra]

  if( atributos_presentes( c("Visa_mlimitecompra", "vm_mlimitecompra") ))
    dataset[, vmr_Visa_mlimitecompra := Visa_mlimitecompra / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldototal", "vm_mlimitecompra") ))
    dataset[, vmr_msaldototal := vm_msaldototal / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldopesos", "vm_mlimitecompra") ))
    dataset[, vmr_msaldopesos := vm_msaldopesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldopesos", "vm_msaldototal") ))
    dataset[, vmr_msaldopesos2 := vm_msaldopesos / vm_msaldototal]

  if( atributos_presentes( c("vm_msaldodolares", "vm_mlimitecompra") ))
    dataset[, vmr_msaldodolares := vm_msaldodolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldodolares", "vm_msaldototal") ))
    dataset[, vmr_msaldodolares2 := vm_msaldodolares / vm_msaldototal]

  if( atributos_presentes( c("vm_mconsumospesos", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumospesos := vm_mconsumospesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mconsumosdolares", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumosdolares := vm_mconsumosdolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_madelantopesos", "vm_mlimitecompra") ))
    dataset[, vmr_madelantopesos := vm_madelantopesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_madelantodolares", "vm_mlimitecompra") ))
    dataset[, vmr_madelantodolares := vm_madelantodolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagado", "vm_mlimitecompra") ))
    dataset[, vmr_mpagado := vm_mpagado / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagospesos", "vm_mlimitecompra") ))
    dataset[, vmr_mpagospesos := vm_mpagospesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagosdolares", "vm_mlimitecompra") ))
    dataset[, vmr_mpagosdolares := vm_mpagosdolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mconsumototal", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumototal := vm_mconsumototal / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagominimo", "vm_mlimitecompra") ))
    dataset[, vmr_mpagominimo := vm_mpagominimo / vm_mlimitecompra]

  # Aqui debe usted agregar sus propias nuevas variables

  # valvula de seguridad para evitar valores infinitos
  # paso los infinitos a NULOS
  infinitos <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.infinite(get(.name)))]
  )

  infinitos_qty <- sum(unlist(infinitos))
  if (infinitos_qty > 0) {
    cat(
      "ATENCION, hay", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n"
    )
    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  # valvula de seguridad para evitar valores NaN  que es 0/0
  # paso los NaN a 0 , decision polemica si las hay
  # se invita a asignar un valor razonable segun la semantica del campo creado
  nans <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.nan(get(.name)))]
  )

  nans_qty <- sum(unlist(nans))
  if (nans_qty > 0) {
    cat(
      "ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    )

    cat("Si no te gusta la decision, modifica a gusto el programa!\n\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }

  cat( "fin AgregarVariables_IntraMes()\n")
}

AgregarVariables_IntraMes(dataset)

## Modelado

### Training Strategy

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 201901, 202107 ]  con un undersampling de los CONTINUA del 2% por un tema de velocidad

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling 
   * training = [ 201901, 202106 ]  con un undersampling de los CONTINUA del 2%  por un tema de velocidad

In [ ]:
GLOBAL_semilla <- 315697

In [ ]:
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria"))
)

set.seed(GLOBAL_semilla, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

dfuture <- dataset[foto_mes==202109]

# undersampling de los CONTINIA al 8%
dataset[, fold_train :=  foto_mes %in% c(202107, 202106,
                                  # 202104, 202103, 
                                  202102, 202101, 
                                  202012, 202011, 202010, 202009, 202008, 202007, 
                                  202006,
                                  202005, 
                                  # 202004, 202003, 
                                  202002, 202001,
                                  201912, 201911,
                                  # 201910 Excluyo por variables rotas
                                  201909, 201908, 201907, 201906,
                                  # 201905  Excluyo por variables rotas
                                  201904, 201903, 201902, 201901
    ) & (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < 0.10 ) ]

dataset[, clase01 := ifelse( clase_ternaria=="CONTINUA", 0, 1 )]

require("lightgbm")

# dejo los datos en el formato que necesita LightGBM
dvalidate <- lgb.Dataset(
  data = data.matrix(dataset[foto_mes==202107, campos_buenos, with = FALSE]),
  label = dataset[foto_mes==202107, clase01],
  free_raw_data = TRUE
)

# aqui se hace la magia informatica con los pesos para poder reutilizar
#  el mismo dataset para training y final_train
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label = dataset[fold_train == TRUE, clase01],
  weight = dataset[fold_train == TRUE, ifelse( 
                                  foto_mes %in% c(202106,
                                  # 202104, 202103, 
                                  202102, 202101, 
                                  202012, 202011, 202010, 202009, 202008, 202007, 
                                  202006,
                                  202005, 
                                  # 202004, 202003, 
                                  202002, 202001,
                                  201912, 201911,
                                  # 201910 Excluyo por variables rotas
                                  201909, 201908, 201907, 201906,
                                  # 201905  Excluyo por variables rotas
                                  201904, 201903, 201902, 201901), 1.0, 0.0)],
  free_raw_data = TRUE
)

rm( dataset )
gc(full = TRUE, verbose= FALSE) # garbage collection

verificacion de datasets de entrenamiento

In [ ]:
nrow( dfuture )
nrow( dvalidate )
nrow( dtrain )

### Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve 

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones intelifgentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 1024]
  * min_data_in_leaf  [64, 8196]

In [ ]:

# parametros basicos del LightGBM
param_basicos <- list(
    objective = "binary",
    metric = "auc",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    force_row_wise = TRUE, # para evitar warning
    seed = GLOBAL_semilla,
    max_bin = 31,
    learning_rate = 0.05,
    feature_fraction = 0.7
)


EstimarGanancia_AUC_lightgbm <- function(x) {

    message(format(Sys.time(), "%a %b %d %X %Y"))
    param_train <- list(
      num_iterations = 2048, # valor grande, lo limita early_stopping_rounds
      early_stopping_rounds = 200
    )

    param_completo <- c(param_basicos, param_train, x)

    modelo_train <- lgb.train(
      data = dtrain,
      valids = list(valid = dvalidate),
      eval = "auc", 
      param = param_completo,
      verbose = -100
    )

    AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

    # esta es la forma de devolver un parametro extra
    attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)
    
    rm(modelo_train)
    gc(full= TRUE, verbose= FALSE)
    
    return(AUC)
}


Complejo seteo de la Bayesian Optimization

In [ ]:
# paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")

el seteo propiamente

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn = EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize = FALSE, # estoy Maximizando AUC
    noisy = FALSE,
    par.set = makeParamSet(
       makeIntegerParam("num_leaves", lower = 8L, upper = 1024L),
       makeIntegerParam("min_data_in_leaf", lower = 64L, upper = 8192L)
    ),
    has.simple.signature = FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time = 600,
    save.file.path = "lineademuerte_2.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters = 20  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type = "se",
    covtype = "matern3_2",
    control = list(trace = TRUE)
)


Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado

In [ ]:
bayesiana_salida <- mbo(obj.fun, learner = surr.km, control = ctrl)

obtengo los mejores hiperparametros

In [ ]:
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
setorder(tb_bayesiana, -y, -num_iterations) # ordeno en forma descendente por AUC = y
mejores_hiperparametros <- tb_bayesiana[1, # el primero es el de mejor AUC
    list(num_leaves, min_data_in_leaf, num_iterations)]

print(mejores_hiperparametros)

## Produccion

### Final Model

magia informatica, ahora  dtrain es [201901, 202107] a todos les asigno el peso en 1.0

In [ ]:
set_field(dtrain, "weight", rep(1.0, nrow(dtrain)))

In [ ]:
param_final <- c(param_basicos, mejores_hiperparametros)

Genero el modelo final

In [ ]:
final_model <- lgb.train(
    data = dtrain,
    param = param_final,
    verbose = -100
)

### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
prediccion <- predict(
  final_model,
  data.matrix(dfuture[, campos_buenos, with = FALSE])
)

### Kaggle

Genero la salida para Kaggle

In [ ]:
# genero la tabla de entrega
tb_entrega <- dfuture[, list(numero_de_cliente)]
tb_entrega[, prob := prediccion]

# ordeno por probabilidad descendente
setorder(tb_entrega, -prob)

# lista de cortes para Predicted
cortes <- c(10000, 10500, 11000, 11500)

# bucle para generar un archivo por cada corte
for (corte in cortes) {
  # crea una copia de la tabla base
  tb_corte <- copy(tb_entrega)
  
  # elimina la columna prob (opcional, puede hacerse después del ordenamiento)
  tb_corte[, prob := NULL]
  
  # asigna Predicted según el corte
  tb_corte[, Predicted := 0L]
  tb_corte[1:corte, Predicted := 1L]
  
  # guarda el archivo con el corte en el nombre
  fwrite(tb_corte, file = paste0("lineademuerte_2_", corte, ".csv"))
}


In [ ]:
format(Sys.time(), "%a %b %d %X %Y")